In [1]:
import pandas as pd
import re
import requests
import json
import functools
from pprint import pprint
from docx import Document
from os import listdir, environ
from os.path import isfile, join, abspath
from collections import defaultdict

In [2]:

datafiles = [f for f in listdir('./data/') if isfile(join('./data/', f))]
columns = ['Дата', 'Адрес', 'Оксид углерода', 'Оксид азота', 'Диоксид азота', 'Диоксид серы', 'Озон', 'Взвешенные частицы PM10', 'Взвешенные частицы PM2.5']

In [3]:
def addDate(fn):
    df = pd.DataFrame(columns=columns)
    with open(abspath('data/'+fn), 'rb') as f:
        pollution_report = Document(f)

        adr_cnt = 0
        for i, paragraph in enumerate(pollution_report.paragraphs):
            row_df = dict.fromkeys(columns)
            adress = re.findall("адресу: (.*?)\\(станции", paragraph.text)
            if adress:
                row_df['Адрес'] = adress[0]
                row_df['Дата']  = fn.split('.')[0]
                for j in range(1, len(pollution_report.tables[adr_cnt].rows)):
                    row = pollution_report.tables[adr_cnt].rows[j]
                    row_text = []
                    for cell in row.cells:
                        row_text.append(cell.text)
                    row_df[row_text[0]] = row_text[1]
                df = df.append(row_df, ignore_index=True)
                adr_cnt += 1
    return df

In [4]:
# Construct dataframe from all of the documents
df = pd.DataFrame(columns=columns)

for file in datafiles:
    df = df.append(addDate(file), ignore_index=True)

In [9]:
# Filter out adresses for better mapbox search
def filter_adress(adr):
    return functools.reduce(lambda x, y: x.replace(y, abbr[y]), abbr, adr).rstrip()

abbr = {'пр.':'проспект', 'ул.': 'улица', 'В.О.': '', 'пер.': 'переулок', 'г.': 'город', 'пос.': 'поселок', 'М.': 'Максима'}
df['Адрес'] = df['Адрес'].map(filter_adress)
adresses = df['Адрес'].unique()
adresses

array(['Московский проспект, дом 19', 'Средний проспект, дом 74',
       'проспект КИМа, дом 26 лит. А', 'улица Карбышева, дом 7',
       'улица Ольги Форш, дом 6', 'город Колпино, Красная улица, дом 1а',
       'поселок Металлострой, Железнодорожная улица, дом 13',
       'Малоохтинский проспект, дом 98', 'Индустриальный проспект, д.64',
       'Уткин проспект, д.16', 'Волхонское шоссе, д.116, корп. 3',
       'город Кронштадт, улица Ильмянинова, дом 4',
       'город Сестрорецк, улица Максима Горького, дом 2',
       'улица Севастьянова, дом 11', 'улица Тельмана, дом 24',
       'улица Профессора Попова, дом 48',
       'город Ломоносов, улица Федюнинского, дом 3',
       'улица Новосельковская, дом 23',
       'город Пушкин, Тиньков переулок, дом 4',
       'Малая Балканская улица, дом 54', 'Шпалерная улица, дом 56',
       'улица Пестеля, дом 1', 'проспект Маршала Жукова, дом 30, корп. 3'],
      dtype=object)

In [10]:
# Get geo-coordinates for each unique adress
bbox = '29.5313, 59.6691, 30.8386, 60.2466'
access_token = 'pk.eyJ1IjoiYWxleDQ5MiIsImEiOiJjank1b3BleXcwOHM4M29tbHY1bWVhaXA1In0.HY6gliRl51T_WMSxdOGbJw'
base_url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?bbox={}&access_token={}"

georef = dict.fromkeys(adresses)

for adress in adresses:
    search_result = requests.get(base_url.format(adress, bbox, access_token)).json()
    pprint(search_result)
    for result in search_result['features']:
        if len(result['properties']) > 0:
            georef.update({adress: result['center']})
            break

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.319603, 59.872354],
               'context': [{'id': 'locality.9901303410134450',
                            'text': 'округ Московская застава'},
                           {'id': 'postcode.4716056166011650',
                            'text': '196105'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                     

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'bbox': [30.310392999964,
                        59.9251709998373,
                        30.3434919997326,
                        59.9375519998175],
               'center': [30.329922, 59.931213],
               'context': [{'id': 'postcode.10517297569016090',
                            'text': '191002'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Лени

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.577333, 59.8113019],
               'context': [{'id': 'locality.655696745997470',
                            'text': 'Металлострой'},
                           {'id': 'postcode.655696746377950', 'text': '196641'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                            'wikidata': 'Q2191'},
            

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.4374322, 59.9319429],
               'context': [{'id': 'locality.383397579243720',
                            'text': 'округ Малая Охта'},
                           {'id': 'postcode.383374014010050', 'text': '195112'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                            'wikidata': 'Q2191'},
       

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.5778495, 59.8083183],
               'context': [{'id': 'locality.655696745997470',
                            'text': 'Металлострой'},
                           {'id': 'postcode.8439403656368600',
                            'text': '196645'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                            'wik

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.3045, 59.9707353],
               'context': [{'id': 'locality.378502722853120',
                            'text': 'округ Чкаловское'},
                           {'id': 'postcode.14615904600779980',
                            'text': '197022'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                            'w

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.2845451, 59.6716225],
               'context': [{'id': 'locality.9900398722717900',
                            'text': 'Пушкин'},
                           {'id': 'postcode.19455198689003660',
                            'text': '196632'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                            'wikidat

{'attribution': 'NOTICE: © 2019 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [30.3479715, 59.9425793],
               'context': [{'id': 'locality.19154498622600990',
                            'text': 'округ Литейный округ'},
                           {'id': 'postcode.18483686972899040',
                            'text': '191028'},
                           {'id': 'place.10100231204958930',
                            'text': 'Санкт-Петербу́рг',
                            'wikidata': 'Q656'},
                           {'id': 'region.6334160928931040',
                            'short_code': 'RU-LEN',
                            'text': 'Ленинградская область',
                     

In [11]:
georef

{'Московский проспект, дом 19': [30.319603, 59.872354],
 'Средний проспект, дом 74': [29.6713605, 60.2004752],
 'проспект КИМа, дом 26 лит. А': [30.2443917, 59.9522399],
 'улица Карбышева, дом 7': [30.327516, 59.939101],
 'улица Ольги Форш, дом 6': [30.3917467, 60.0427613],
 'город Колпино, Красная улица, дом 1а': [30.4220037, 59.7130928],
 'поселок Металлострой, Железнодорожная улица, дом 13': [30.577333,
  59.8113019],
 'Малоохтинский проспект, дом 98': [30.3996642, 59.9297239],
 'Индустриальный проспект, д.64': [30.4660558, 59.9522512],
 'Уткин проспект, д.16': [30.4374322, 59.9319429],
 'Волхонское шоссе, д.116, корп. 3': [30.3391245, 59.7344552],
 'город Кронштадт, улица Ильмянинова, дом 4': [30.3365582, 59.9227183],
 'город Сестрорецк, улица Максима Горького, дом 2': [30.5778495, 59.8083183],
 'улица Севастьянова, дом 11': [30.3269783, 59.8749832],
 'улица Тельмана, дом 24': [30.487488, 59.893182],
 'улица Профессора Попова, дом 48': [30.3045, 59.9707353],
 'город Ломоносов, улиц

In [12]:
df['lon'] = df['Адрес'].apply(lambda row: georef[row][0])
df['lat'] = df['Адрес'].apply(lambda row: georef[row][1])

In [13]:
def numberIt(cell):
    if not cell or '*' in cell:
        return None
    tokens = cell.split(' ')
    if ('менее' in tokens):
        return float(tokens[-1])/2
    else:
        return float(cell)
    
df.iloc[:,2:9] = df.iloc[:,2:9].applymap(numberIt)

In [14]:
df.to_csv('spb_pollution_record.csv')